In [1]:
%pip install langchain

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

#openai.api_key  = os.environ['OPENAI_API_KEY']

In [3]:
%pip install pypdf 

                                              0.0/256.1 kB ? eta -:--:--
     ----                                    30.7/256.1 kB 1.3 MB/s eta 0:00:01
     -----                                 41.0/256.1 kB 326.8 kB/s eta 0:00:01
     --------                              61.4/256.1 kB 465.5 kB/s eta 0:00:01
     --------                              61.4/256.1 kB 465.5 kB/s eta 0:00:01
     --------                              61.4/256.1 kB 465.5 kB/s eta 0:00:01
     ---------------                      112.6/256.1 kB 344.8 kB/s eta 0:00:01
     ---------------                      112.6/256.1 kB 344.8 kB/s eta 0:00:01
     -----------------                    122.9/256.1 kB 277.4 kB/s eta 0:00:01
     --------------------                 143.4/256.1 kB 293.9 kB/s eta 0:00:01
     ---------------------                153.6/256.1 kB 270.0 kB/s eta 0:00:01
     ------------------------             174.1/256.1 kB 299.5 kB/s eta 0:00:01
     ------------------------             174.1


[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [37]:
from langchain.document_loaders import PyPDFLoader
loaders =[ 
    PyPDFLoader("D:\Documents\CDC Internship\ChatBot\Contact Us.pdf"),
    PyPDFLoader("D:\Documents\CDC Internship\ChatBot\Overview.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [38]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 150,
    chunk_overlap = 15
)

In [39]:
splits = text_splitter.split_documents(docs)

In [40]:
len(splits)

62

In [42]:
splits[34].page_content[:100]

'functions to the  Mutual Fund industry. ( Learn more ) \nCentralized Information Sharing Solution for'

In [36]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass  `openai_api_key` as a named parameter. (type=value_error)

In [3]:
from langchain.vectorstores import Chroma

In [35]:
persist_directory= 'D:\Documents\chroma'
!rm -rf ./Documents/chroma
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embedding,
    persist_directory=persist_directory
)
print(vectordb.collection_count())


'rm' is not recognized as an internal or external command,
operable program or batch file.


NameError: name 'embedding' is not defined